In [11]:
import pandas as pd

def compute_stats(df, freq_label):
    mean_val = df['CASES'].mean()
    var_val = df['CASES'].var()
    ratio = var_val / mean_val if mean_val != 0 else None
    return {
        "Frequência": freq_label,
        "Média (μ)": round(mean_val, 2),
        "Variância (σ²)": round(var_val, 2),
        "σ²/μ": round(ratio, 2) if ratio else "N/A"
    }

# ---------------------------------------------------------
# 1. Ler dados
# ---------------------------------------------------------
paths = {
    'RN': '/home/rgarcia/mcs/data/processed/sinan/DENG240810.parquet',
    'RJ': '/home/rgarcia/mcs/data/processed/sinan/DENG33.parquet'
}

result_rows = []

for region, path in paths.items():
    df = pd.read_parquet(path)
    df['DT_NOTIFIC'] = pd.to_datetime(df['DT_NOTIFIC'], errors='coerce')
    df = df.dropna(subset=['DT_NOTIFIC'])

    # ----------------------------------------
    # Diário
    daily = df.groupby('DT_NOTIFIC')['CASES'].sum().reset_index()
    stats_daily = compute_stats(daily, 'Diário')
    stats_daily['Região'] = region
    result_rows.append(stats_daily)

    # ----------------------------------------
    # Semanal
    df['Ano'] = df['DT_NOTIFIC'].dt.year
    df['SemanaEpi'] = df['DT_NOTIFIC'].dt.isocalendar().week
    weekly = df.groupby(['Ano', 'SemanaEpi'])['CASES'].sum().reset_index()
    stats_weekly = compute_stats(weekly, 'Semanal')
    stats_weekly['Região'] = region
    result_rows.append(stats_weekly)

# ---------------------------------------------------------
# 2. Gerar tabela final
# ---------------------------------------------------------
result_df = pd.DataFrame(result_rows)[['Região', 'Frequência', 'Média (μ)', 'Variância (σ²)', 'σ²/μ']]
print(result_df.to_string(index=False))


Região Frequência  Média (μ)  Variância (σ²)   σ²/μ
    RN     Diário       9.42          323.22  34.30
    RN    Semanal      65.68        12138.26 184.80
    RJ     Diário      44.59         6030.20 135.24
    RJ    Semanal     310.96       218018.94 701.12
